In [1]:
import pandas as pd
from pandasql import sqldf
import numpy as np
from datetime import datetime

### Текст задания:

1) Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Дана таблицs peas:

* st_id(int) - ID ученика
* timest(timestamp) - Время решения карточки
* correct(bool) - Правильно ли решена горошина?
* subject(text) - Дисциплина, в которой находится горошина

studs:

* st_id(int) - ID ученика
* test_grp(text) - Метка ученика в данном эксперименте

и checks:

* st_id(int) - ID ученика
* sale_time(timestamp) - Время покупки
* money(int) - Цена, по которой приобрели данный курс
* subject(text) - Дисциплина, на которую приобрели полный доступ

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

1. ARPU 
2. ARPAU
3. ARPPU
4. CR в покупку 
5. СR активного пользователя в покупку 
6. CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

##### Чтобы проверить правильность запроса попробуем сгенерировать интересующие нас данные и проверить написанный запрос с помощью библиотеки Pandasql

#### Сначала создадим таблицу peas

In [2]:
# Создаем id пользователей от 1 до 1000
all_user_ids = np.arange(1, 1001)

In [3]:
n = 10000

In [4]:
# Создаем данные колонки st_id путем случайного выбора
st_id = np.random.choice(all_user_ids, n)

In [5]:
#times = pd.timedelta_range(start='30 sec', end='20 min', freq='1s') \
        #.total_seconds()
#timedelta_ts = np.random.choice(times, n)

In [6]:
DT_FMT = "%d/%m/%Y:%H:%M:%S"

def get_date_range(dt_from, dt_to, freq="1D", dt_fmt=DT_FMT, **kwargs):
    """
    Функция создает список дат в формате timestampб принимая на вход следующие параметры:
    dt_from - дата начала периода
    dt_to - дата окончания периода
    freq - периодичность генерируемых дат
    dt_fmt - формат даты
    """
    dt_from = pd.to_datetime(dt_from, format=dt_fmt)
    dt_to = pd.to_datetime(dt_to, format=dt_fmt)
    date_range = pd.date_range(dt_from, dt_to, freq=freq, **kwargs)
    date_timestamp = map(datetime.timestamp, date_range) # переводим в timestamp
    all_timest = list(date_timestamp)
    return all_timest

In [7]:
# Генерируем перечень дат в заданном промежутке
all_solut_time = get_date_range("10/03/2020:00:00:05", "20/04/2020:23:59:00", freq="30S")
# Выбираем случайным образом список дат длиной n
timest = np.random.choice(all_solut_time, n)

In [8]:
# Генерируем перечень учебных дисциплин
all_subject = ['math', 'SQL', 'python', 'BI', 'product analysis']
# Выбираем случайным образом список учебных дисциплин длиной n
subject = np.random.choice(all_subject, n)

In [9]:
user_actions = pd.DataFrame({'st_id': st_id, 
                             'timest': timest,
                             'subject': subject})

In [10]:
user_actions

,st_id,timest,subject
0,128,1.583792e+09,BI
1,932,1.586986e+09,product analysis
2,561,1.585573e+09,math
3,113,1.585243e+09,SQL
4,725,1.584687e+09,SQL
...,...,...,...
9995,332,1.584515e+09,BI
9996,910,1.585367e+09,SQL
9997,984,1.585777e+09,product analysis
9998,306,1.585462e+09,python


In [11]:
correct = 0.7
def generate_funel_correct(st_id, timest, subject, correct):
    """
    Функция генерирует случайным образом отметку о правильности выполнения задания, учитывая следующие парметры:
    st_id - id студента, сгенерированные ранее
    timest - время решения задачи в timestamp, сгенерированное ранее
    subject - список учебных дисциплин
    correct - вероятность правильного решения задания
    """
          
    df = pd.DataFrame()
    
    if np.random.binomial(1, correct, 1)[0]:
        df = pd.DataFrame({
                           'st_id': st_id, 
                           'timest': timest,
                           'subject': subject,
                           'correct': True}, index=[0])
    else:
        df = pd.DataFrame({
                           'st_id': st_id, 
                           'timest': timest,
                           'subject': subject,
                           'correct': False}, index=[0])
        
    return df

In [12]:
# Создаем пустой df
peas = pd.DataFrame()

In [13]:
# Итерируемся по df и запускаем функцию генерации ответов для каждой строки
for index, row in user_actions.iterrows():
    user_df = generate_funel_correct(row['st_id'], row['timest'], row['subject'], correct=0.7)
    peas = peas.append(user_df)

In [14]:
peas

,st_id,timest,subject,correct
0,128,1.583792e+09,BI,True
0,932,1.586986e+09,product analysis,True
0,561,1.585573e+09,math,False
0,113,1.585243e+09,SQL,False
0,725,1.584687e+09,SQL,False
...,...,...,...,...
0,332,1.584515e+09,BI,True
0,910,1.585367e+09,SQL,True
0,984,1.585777e+09,product analysis,False
0,306,1.585462e+09,python,False


#### Создадим таблицу studs

Перечень st_id у нас уже есть. Проставим метку участия ученика в эксперименте

In [15]:
dev = 0.5
def generate_funel_grp(st_id, dev):
    """
    Функция генерирует случайным образом отметку об участии ученика в эксперименте, учитывая следующие парметры:
    st_id - id студента, сгенерированные ранее
    dev - вероятность деления студентов на тестовую и контрольную группы
    """
          
    df = pd.DataFrame()
    
    if np.random.binomial(1, dev, 1)[0]:
        df = pd.DataFrame({
                           'st_id': st_id, 
                           'test_grp': 'test'}, index=[0])
    else:
        df = pd.DataFrame({
                           'st_id': st_id, 
                           'test_grp': 'control'}, index=[0])
        
    return df

In [16]:
user_grp = pd.DataFrame({'st_id': all_user_ids})

In [17]:
# Создаем пустой df
studs = pd.DataFrame()

In [18]:
# Итерируемся по df и запускаем функцию генерации группы для каждой строки
for index, row in user_grp.iterrows():
    group_df = generate_funel_grp(row['st_id'], dev=0.5)
    studs = studs.append(group_df)

#### Создадим таблицу checks

Эта таблица посвящена тем студентам, которые проходят курс платно, поэтому n будет меньше.

In [19]:
n = 600

In [20]:
# Создаем данные колонки st_id путем случайного выбора
st_id = np.random.choice(studs.st_id, n)

In [21]:
# Генерируем перечень дат в заданном промежутке
all_buying_time = get_date_range("10/03/2020:00:00:05", "20/04/2020:23:59:00", freq="5M")
# Выбираем случайным образом список дат длиной n
timest = np.random.choice(all_buying_time, n)

In [22]:
# Генерируем перечень стоимостей курса
all_money = [50, 75, 85, 100]
# Выбираем случайным образом список учебных дисциплин длиной n
money = np.random.choice(all_money, n)

In [23]:
# Генерируем перечень учебных дисциплин
all_subject = ['math', 'SQL', 'python', 'BI', 'product analysis']
# Выбираем случайным образом список учебных дисциплин длиной n
subject = np.random.choice(all_subject, n)

In [24]:
checks = pd.DataFrame({'st_id': st_id, 
                      'sale_time': timest,
                      'money': money, 
                      'subject': subject})

Напишем запрос

In [25]:
SQL_1 = """
     SELECT
        test_grp,
        sum(ch.money) / count(DISTINCT st.st_id)                               AS ARPU,
        sum(ch.money) / count(DISTINCT ch.st_id)                               AS ARPPU,
        (count(DISTINCT ch.st_id) * 100 / count(DISTINCT st.st_id))            AS CR_to_purchase,
        (count(DISTINCT ch.st_id) * 100 / count(DISTINCT p.st_id))             AS CR_active_user_to_purchase,
        count(DISTINCT CASE WHEN p.subject = 'math' AND ch.subject = 'math' THEN st.st_id END) * 100 
        / count(DISTINCT CASE WHEN p.subject = 'math' THEN st.st_id END)       AS CR_math
    FROM peas AS p                                                                 -- все пользователи в эксперименте
    LEFT JOIN studs AS st ON p.st_id = st.st_id                                        -- активные пользователи
    LEFT JOIN checks AS ch ON p.st_id = ch.st_id                                    -- платящие пользователи
    GROUP BY test_grp
    """

In [26]:
sqldf(SQL_1)

,test_grp,ARPU,ARPPU,CR_to_purchase,CR_active_user_to_purchase,CR_math
0,control,439,1014,43,43,9
1,test,504,1045,48,48,12


##### Проверим в pandas

In [27]:
common_df = peas.merge(studs, on='st_id', how='left') \
            .merge(checks, on='st_id', how='left')
common_df.head()

,st_id,timest,subject_x,correct,test_grp,sale_time,money,subject_y
0,128,1.583792e+09,BI,True,test,NaN,NaN,NaN
1,932,1.586986e+09,product analysis,True,test,1.585595e+09,85.0,SQL
2,561,1.585573e+09,math,False,test,NaN,NaN,NaN
3,113,1.585243e+09,SQL,False,test,1.585595e+09,50.0,SQL
4,725,1.584687e+09,SQL,False,control,1.585595e+09,85.0,python


In [28]:
metrics = pd.DataFrame(columns={
                        'all_users',
                        'paying_users',
                        'ARPU',
                        'ARPPU',
                        'CR_to_purchase',
                        'CR_active_user_to_purchase',
                        'CR_math'}, 
                       index=['control', 'test'])

In [29]:
active_math = common_df.query('subject_x == "math"') \
                       .groupby('test_grp') \
                       .st_id \
                       .nunique()

purchase_math = common_df.query('subject_y == "math" and subject_x == "math"') \
                         .groupby('test_grp') \
                         .st_id \
                         .nunique()

In [30]:
metrics['all_users'] = common_df.groupby('test_grp').st_id.nunique().values
metrics['paying_users'] = common_df.query('money > 0') \
                                   .groupby('test_grp') \
                                   .agg({'st_id': 'nunique'}).values
metrics['active_users'] = common_df.query('timest > 0') \
                                   .groupby('test_grp') \
                                   .agg({'st_id': 'nunique'}).values
metrics['revenue'] = common_df.groupby('test_grp').agg({'money': 'sum'}).values
metrics['ARPU'] = metrics.revenue / metrics.all_users
metrics['ARPPU'] = metrics.revenue / metrics.paying_users
metrics['CR_to_purchase'] = metrics.paying_users / metrics.all_users * 100
metrics['CR_active_user_to_purchase'] = metrics.paying_users / metrics.active_users * 100
metrics['CR_math'] = purchase_math * 100 / active_math 

In [31]:
metrics

,all_users,CR_to_purchase,ARPPU,CR_math,CR_active_user_to_purchase,paying_users,ARPU,active_users,revenue
control,501,43.313373,1014.953917,9.728507,43.313373,217,439.610778,501,220245.0
test,499,48.296593,1045.000000,12.296984,48.296593,241,504.699399,499,251845.0


1) Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной". Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин за час.
Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов за март 2020 года.

Дана таблица peas:

* st_id(int) - ID ученика
* timest(timestamp) - Время решения карточки
* correct(bool) - Правильно ли решена горошина?
* subject(text) - Дисциплина, в которой находится горошина

Напишем данный запрос для Clickhouse. 

In [32]:
SQL_2 = """
    SELECT DISTINCT st_id
    FROM
    (
        SELECT
            st_id,
            COUNT(*) AS peas
        FROM peas

        WHERE correct = True
          AND toMonth(timest) = 3
          AND toYear(timest)  = 2020

        GROUP BY st_id, date_trunc('hour', timest)
        HAVING COUNT(*) > 20
    )
"""